In [1]:
import sys
import os
import csv
import numpy as np
import torch

In [2]:
sys.path.append(os.path.join(sys.path[0], '../../../'))

In [3]:
from endure.lcm.data.generator import LCMDataGenerator
from endure.data.io import Reader
from jobs.bayesian_pipeline import BayesianPipeline
from endure.lsm.solver.classic_solver import ClassicSolver
from endure.lsm.cost import EndureCost
from endure.lsm.types import LSMDesign, System, Policy, Workload


In [4]:
config = Reader.read_config("../../../endure.toml")
bayesian_optimizer = BayesianPipeline(config)
generator = LCMDataGenerator(config)
solver = ClassicSolver(config)
cf = EndureCost(config)

In [5]:
def find_analytical_results(system, z0, z1, q, w):
        nominal_design, nominal_solution = solver.get_nominal_design(system, z0, z1, q, w)
        x = np.array([[nominal_design.h, nominal_design.T]])
        train_x = torch.tensor(x, dtype=torch.float64)
        policy = nominal_design.policy
        cost = solver.nominal_objective(x[0], policy, system, z0, z1, q, w)
        train_y = torch.tensor(cost, dtype=torch.float64).unsqueeze(-1)
        return nominal_design, cost

In [6]:
def update_best_designs(best_designs, new_x: torch.Tensor, new_y: torch.Tensor) -> None:
        for x, y in zip(new_x, new_y):
            best_designs.append((LSMDesign(x[0].item(), np.ceil(x[1].item()), Policy.Leveling), y.item()))


In [7]:
def generate_initial_data(z0, z1, q, w, system: System, n: int = 30):
        train_x = []
        train_y = []
        for _ in range(n):
            design = generator._sample_design(system)
            x_values = np.array([design.h, design.T])
            # cost is negated here
            cost = cf.calc_cost(design, system, z0, z1, q, w)
            train_x.append(x_values)
            train_y.append(cost)
        train_x = np.array(train_x)
        train_x = torch.tensor(train_x, dtype=torch.float64)
        train_y = torch.tensor(train_y, dtype=torch.float64).unsqueeze(-1)
        best_y = train_y.min().item()
        return train_x, train_y, best_y

In [8]:
acqf = "ExpectedImprovement"

In [9]:
def run(system, z0, z1, q, w):
    best_designs = []
    print(system)
    h_bounds = torch.tensor([config["job"]["BayesianOptimization"]["bounds"]["h_min"],
                                      system.H])
    T_bounds = torch.tensor([config["job"]["BayesianOptimization"]["bounds"]["T_min"],
                                      config["job"]["BayesianOptimization"]["bounds"]["T_max"]])
    bounds = torch.stack([h_bounds, T_bounds])
    train_x, train_y, best_y = generate_initial_data(z0, z1, q, w, system, 50)
    for i in range(15):
        
        new_candidates = bayesian_optimizer.get_next_points(train_x, train_y,
                                                  best_y, bounds, acqf, 1)
        new_designs = [LSMDesign(x[0].item(), np.ceil(x[1].item()), Policy.Leveling) for x in new_candidates]
        new_target = torch.tensor([cf.calc_cost(design, system, z0, z1, q, w) for design in new_designs])
        new_target = new_target.unsqueeze(-1)
        train_x = torch.cat([train_x, new_candidates])
        train_y = torch.cat([train_y, new_target])
        best_y = train_y.min().item()
        update_best_designs(best_designs, new_candidates, new_target)
    sorted_designs = sorted(best_designs, key=lambda x: x[1])
    if sorted_designs:
        best_design, best_cost = sorted_designs[0]
        return best_design, best_cost
    else:
        return None, None

In [26]:
def compare_designs(n_runs=10, csv_filename='design_comparison.csv'):
        with open(csv_filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Entries per page(E)','Physical Entries per page(B)', 'Selectivity(s)', 'Max bits per element(H)', 'Total elements (N)', 'Empty Reads', 'Non-Empty Reads', 'Range Queries', 'Writes', 'BO Design', 'Analytical Design','BO Cost', 'Analytical Cost', 'Diff(Analytical-Bayesian)'])
            #TODO if this works then put this in toml file
            for i in range(n_runs):
                print(f"Iteration {i+1}/{n_runs} running")
                system = generator._sample_system()
                z0, z1, q, w = generator._sample_workload(4)
                print(np.floor(system.H))
                bo_design, bo_cost = bayesian_optimizer.run(system, z0, z1, q, w)
                analytical_design, analytical_cost = find_analytical_results(system, z0, z1, q, w)
                writer.writerow([system.E, system.B, system.s, system.H, system.N, z0, z1, q, w, bo_design, analytical_design, bo_cost, analytical_cost, analytical_cost-bo_cost])

In [28]:
compare_designs()

Iteration 1/10 running
17.0
[ 9.478 17.     0.   ]
[16.124 24.     0.   ]
[10.193 10.     0.   ]
[ 5.768 27.     0.   ]
[ 4.539 26.     0.   ]
[11.091 13.     0.   ]
[ 2.795 27.     1.   ]
[ 7.527 13.     1.   ]
[16.659 10.     1.   ]
[15.881 18.     1.   ]
[ 5.15 23.    1.  ]
[9.169 8.    0.   ]
[ 8.96 26.    0.  ]
[15.402 29.     1.   ]
[ 6.55 16.    0.  ]
[ 4.367 22.     0.   ]
[12.404 10.     1.   ]
[ 9.912 18.     0.   ]
[0.966 3.    1.   ]
[5.464 3.    0.   ]
[ 4.868 25.     1.   ]
[13.541 17.     1.   ]
[3.408 6.    0.   ]
[14.088 11.     1.   ]
[16.631  6.     1.   ]
[ 7.699 28.     1.   ]
[10.923 27.     1.   ]
[12.494  6.     0.   ]
[ 8.264 24.     1.   ]
[9.908 8.    0.   ]


/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(
/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(
/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(
/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCa

new_candidates tensor([[ 5.9954, 24.0000,  0.0000]], dtype=torch.float64)
new target tensor([[0.7940]])
new_candidates tensor([[ 1., 11.,  0.]], dtype=torch.float64)
new target tensor([[0.9043]])
new_candidates tensor([[9.7226, 5.0000, 0.0000]], dtype=torch.float64)
new target tensor([[1.1461]])
new_candidates tensor([[ 1., 25.,  0.]], dtype=torch.float64)
new target tensor([[0.8102]])
new_candidates tensor([[1., 2., 0.]], dtype=torch.float64)
new target tensor([[2.2954]])
new_candidates tensor([[10.1456, 12.0000,  0.0000]], dtype=torch.float64)
new target tensor([[0.9057]])
new_candidates tensor([[10.1866, 21.0000,  0.0000]], dtype=torch.float64)
new target tensor([[0.8430]])
new_candidates tensor([[1., 9., 0.]], dtype=torch.float64)
new target tensor([[0.9465]])
new_candidates tensor([[12.4686, 24.0000,  0.0000]], dtype=torch.float64)
new target tensor([[0.8727]])
new_candidates tensor([[ 6.8153, 30.0000,  0.0000]], dtype=torch.float64)
new target tensor([[0.7921]])
new_candidates te

/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)


new_candidates tensor([[ 1., 23.,  0.]], dtype=torch.float64)
new target tensor([[1.3992]])


...

new_candidates tensor([[ 5.0745, 32.0000,  0.0000]], dtype=torch.float64)
new target tensor([[1.5033]])


/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(
/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


new_candidates tensor([[ 5.0648, 19.0000,  0.0000]], dtype=torch.float64)
new target tensor([[1.4716]])


/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(
/Users/anweshasaha/Desktop/midas-projects/endure-torch/venv_endure_torch/lib/python3.9/site-packages/botorch/optim/initializers.py:404: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


new_candidates tensor([[1.0014, 9.0000, 0.0000]], dtype=torch.float64)
new target tensor([[1.6792]])
Best Design Found:
Design: h=5.064832873748285, T=19.0, Policy=Policy.Leveling, Cost=1.471623420715332
Cost for the nominal design using analytical solver:  1.4635139281966993
Nominal Design suggested by analytical solver:  LSMDesign(h=4.037920889188412, T=19.394919633123656, policy=<Policy.Leveling: 1>, Q=1.0, Y=1.0, Z=1.0, K=[])
Iteration 10/10 running
19.0
[12.444 21.     0.   ]
[18.128 26.     1.   ]
[10.616 11.     1.   ]
[ 3.081 10.     1.   ]
[14.39 14.    0.  ]
[5.742 8.    1.   ]
[16.614 23.     1.   ]
[ 4.778 11.     0.   ]
[16.916 21.     1.   ]
[6.313 3.    1.   ]
[ 6.055 18.     1.   ]
[ 3.548 28.     1.   ]
[15.621  6.     0.   ]
[15.618 29.     1.   ]
[13.676 14.     0.   ]
[13.115  9.     1.   ]
[ 7.016 21.     1.   ]
[ 6.115 11.     0.   ]
[ 6.36 29.    1.  ]
[ 7.815 14.     1.   ]
[ 9.409 10.     1.   ]
[1.367 7.    1.   ]
[9.984 5.    1.   ]
[15.337 30.     1.   ]
[ 4

In [ ]:
cf.calc_cost(LSMDesign(h=15.83, T=31.0, policy=Policy.Leveling, Q=1.0, Y=1.0, Z=1.0, K=[]), System(E=1024, s=2.9102648491798314e-8, B=32.0, N=526728795, H=15.847837338766473, phi=1.0), 0.09, 0.205, 0.605, 0.09999999999999998)

In [17]:
df[df['Diff(Analytical-Bayesian)'] > -0.2].shape[0] / df.shape[0]

1.0

In [18]:
df.columns

Index(['Entries per page(E)', 'Physical Entries per page(B)', 'Selectivity(s)',
       'Max bits per element(H)', 'Total elements (N)', 'Empty Reads',
       'Non-Empty Reads', 'Range Queries', 'Writes', 'BO Design',
       'Analytical Design', 'BO Cost', 'Analytical Cost',
       'Diff(Analytical-Bayesian)', 'norm_delta'],
      dtype='object')

In [19]:
df.sort_values(by='norm_delta')

,Entries per page(E),Physical Entries per page(B),Selectivity(s),Max bits per element(H),Total elements (N),Empty Reads,Non-Empty Reads,Range Queries,Writes,BO Design,Analytical Design,BO Cost,Analytical Cost,Diff(Analytical-Bayesian),norm_delta
2,1024,64.0,1.578232e-08,15.183822,371473242,0.724,0.116,0.067,0.093,"LSMDesign(h=7.0208520822663765, T=25.0, policy...","LSMDesign(h=9.122361650949996, T=22.7374994481...",0.307190,0.297559,-0.009631,-0.032366
5,2048,64.0,2.690755e-08,11.492196,985130846,0.298,0.014,0.080,0.608,"LSMDesign(h=7.686885772440743, T=9.0, policy=<...","LSMDesign(h=5.837562636760175, T=8.23091799545...",0.531936,0.517104,-0.014832,-0.028683
9,4096,8.0,8.861546e-09,19.049582,117027274,0.402,0.042,0.536,0.020,"LSMDesign(h=8.730164831599005, T=32.0, policy=...","LSMDesign(h=5.850761350758528, T=31.0, policy=...",1.182491,1.165189,-0.017302,-0.014849
8,4096,32.0,7.840913e-08,6.699340,838139016,0.007,0.371,0.539,0.083,"LSMDesign(h=1.3249983487381576, T=31.0, policy...","LSMDesign(h=0.0, T=31.0, policy=<Policy.Leveli...",2.686328,2.656574,-0.029754,-0.011200
3,1024,128.0,4.006619e-08,18.649666,618309997,0.254,0.419,0.301,0.026,"LSMDesign(h=9.100760402407493, T=32.0, policy=...","LSMDesign(h=6.305051127539037, T=31.0, policy=...",0.896752,0.888628,-0.008124,-0.009142
0,8192,16.0,3.101181e-09,5.971071,361670612,0.328,0.127,0.183,0.362,"LSMDesign(h=2.667716675677109, T=8.0, policy=<...","LSMDesign(h=3.0023291538887427, T=8.2692225457...",1.654076,1.650681,-0.003395,-0.002057
6,1024,64.0,8.181968e-08,11.096073,810591713,0.703,0.115,0.162,0.020,"LSMDesign(h=7.660157495080137, T=31.0, policy=...","LSMDesign(h=7.156997604714947, T=31.0, policy=...",0.588504,0.587334,-0.001170,-0.001992
4,1024,64.0,4.197560e-08,6.229658,876405779,0.187,0.307,0.455,0.051,"LSMDesign(h=2.755166082010438, T=31.0, policy=...","LSMDesign(h=2.3268195382363333, T=30.999999999...",1.423774,1.421409,-0.002365,-0.001664
7,8192,8.0,7.793652e-08,7.352159,126275060,0.172,0.194,0.619,0.015,"LSMDesign(h=3.1054661799798233, T=32.0, policy...","LSMDesign(h=2.043814481339874, T=31.0, policy=...",2.483838,2.480531,-0.003307,-0.001333
1,2048,64.0,4.375817e-08,8.834162,233438177,0.288,0.617,0.022,0.073,"LSMDesign(h=7.188063386900873, T=16.0, policy=...","LSMDesign(h=6.908008315273583, T=14.3590380161...",0.737131,0.736479,-0.000652,-0.000885


In [22]:
df.iloc[2]['BO Design']

'LSMDesign(h=7.0208520822663765, T=25.0, policy=<Policy.Leveling: 1>, Q=1.0, Y=1.0, Z=1.0, K=[])'

In [23]:
df.iloc[2]['Analytical Design']

'LSMDesign(h=9.122361650949996, T=22.737499448126428, policy=<Policy.Leveling: 1>, Q=1.0, Y=1.0, Z=1.0, K=[])'

In [ ]:
df.iloc[36]['Max bits per element(H)']

In [ ]:
K=2
bounds=torch.tensor([[0.0] * 3, [1.0] * 2 + [K-1]]),


In [ ]:
bounds

In [ ]:
k = 2
lower_bounds = torch.tensor([0.0, 2.0, 0])
upper_bounds = torch.tensor([10, 32, 1])
bounds = torch.stack([lower_bounds, upper_bounds])
bounds

In [ ]:
h_bounds = torch.tensor([0, 10])
t_bounds = torch.tensor([2, 32])
policy_bounds = torch.tensor([0.0, 1.0])
bounds = torch.stack([h_bounds, t_bounds, policy_bounds], dim=-1)
bounds

In [ ]:
x=torch.tensor([[0.7158, 2.0000, 0.0000]])
continuous_data = x[:, :2]
categorical_data = x[:, 2:]
scaled_continuous_data = (continuous_data - bounds[:, :2][0]) / (bounds[:, :2][1] - bounds[:, :2][0])
scaled_data = torch.cat([scaled_continuous_data, categorical_data], dim=-1)
scaled_data